# Projet 5 : Segmenter des clients d'un site e-commerce

*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *Data Scientist* d'OpenClassroooms.

L'objectif pricipal est de réaliser **une segmentation des clients** d'un site de e-commerce, **une proposition de contrat de maintenance** devra être inclue.

Les données mises à notre disposition proviennent du site *kaggle* :
https://www.kaggle.com/olistbr/brazilian-ecommerce

# Partie I : Data Wrangling

L'objectif de ce notebook est de décrire les opérations de nettoyage nécessaires à l'obtention d'un jeu de données exploitable.

In [1]:
# Import usual libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Récupération-des-données" data-toc-modified-id="Récupération-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Récupération des données</a></span></li><li><span><a href="#Fusion-des-données" data-toc-modified-id="Fusion-des-données-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fusion des données</a></span></li><li><span><a href="#Ingénierie-des-variables" data-toc-modified-id="Ingénierie-des-variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ingénierie des variables</a></span><ul class="toc-item"><li><span><a href="#Variables-RFM" data-toc-modified-id="Variables-RFM-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Variables RFM</a></span></li><li><span><a href="#Nombre-de-produits-par-panier" data-toc-modified-id="Nombre-de-produits-par-panier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Nombre de produits par panier</a></span></li><li><span><a href="#Variables-liées-aux-produits" data-toc-modified-id="Variables-liées-aux-produits-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Variables liées aux produits</a></span></li><li><span><a href="#Catégories-des-produits" data-toc-modified-id="Catégories-des-produits-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Catégories des produits</a></span></li><li><span><a href="#Variables-liées-aux-commentaires" data-toc-modified-id="Variables-liées-aux-commentaires-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Variables liées aux commentaires</a></span></li><li><span><a href="#Moyens-de-paiement" data-toc-modified-id="Moyens-de-paiement-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Moyens de paiement</a></span></li></ul></li><li><span><a href="#Mise-à-l'échelle-des-données" data-toc-modified-id="Mise-à-l'échelle-des-données-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Mise à l'échelle des données</a></span></li><li><span><a href="#Création-d'un-pipeline" data-toc-modified-id="Création-d'un-pipeline-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Création d'un pipeline</a></span></li></ul></div>

## Récupération des données

Une fois les données téléchargées, nous pouvons les charger dans un DataFrame en utilisant la librairie **pandas**.

In [3]:
df_customers =  pd.read_csv("data/olist_customers_dataset.csv")
df_geolocation = pd.read_csv("data/olist_geolocation_dataset.csv")
df_order_items = pd.read_csv("data/olist_order_items_dataset.csv")
df_order_payments = pd.read_csv("data/olist_order_payments_dataset.csv")
df_order_reviews = pd.read_csv("data/olist_order_reviews_dataset.csv")
df_orders = pd.read_csv("data/olist_orders_dataset.csv")
df_products = pd.read_csv("data/olist_products_dataset.csv")
df_sellers = pd.read_csv("data/olist_sellers_dataset.csv")
df_translation = pd.read_csv("data/product_category_name_translation.csv")

Regardons ce qui est contenu dans chaque DataFrame.

In [4]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [5]:
df_geolocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


In [6]:
df_order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [7]:
df_order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [8]:
df_order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   review_id                100000 non-null  object
 1   order_id                 100000 non-null  object
 2   review_score             100000 non-null  int64 
 3   review_comment_title     11715 non-null   object
 4   review_comment_message   41753 non-null   object
 5   review_creation_date     100000 non-null  object
 6   review_answer_timestamp  100000 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [9]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [10]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [11]:
df_sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [12]:
df_translation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


Nous pouvons noter plusieurs choses :
1. Il va falloir fusionner ces tables afin de créer une table unique
2. La table geolocation n'apporte rien de plus que la table *customers*
3. Beaucoup de données redondantes
4. Les tables contiennent très peu de données manquantes

## Fusion des données

Avant de commencer à nettoyer les données, nous allons commencer par fusionner les données afin d'obtenir une table unique. Ce qui sera bien plus pratique pour manipuler les données.  
Nous en profiterons pour convertir les dates dans un format date.

In [13]:
def merge_data(list_df):
    """Function merging different DataFrames to get a unique one.
     Parameters:
    list_df: list object
        a list with all dataframes to be merged
    -----------
    Return:
        DataFrame
    """
    df = (pd.merge(left=list_df[0], right=list_df[1],
                   how="left", on="customer_id")
            .merge(right=list_df[2], how="left",
                   on="order_id", copy=False)
            .merge(right=list_df[3], how="left",
                   on="product_id", copy=False)
            .merge(right=list_df[4], how="left",
                   on="order_id", copy=False)
            .merge(right=list_df[5], how="left",
                   on="order_id", copy=False)
            .merge(right=list_df[6], how="left",
                   on="product_category_name", copy=False))
    return df

In [14]:
list_df = [df_customers,
           df_orders,
           df_order_items,
           df_products,
           df_order_payments,
           df_order_reviews,
           df_translation]

df = merge_data(list_df)

In [15]:
def date_monthly(data, feat_to_convert):
    """Function converting data features into a monthy basis date.
    Parameters:
    data: DataFrame
        the pandas object holding data
    feat_to_convert: list of strings
        list holding the name of features to convert
    -----------
    Return:
        DataFrame
    """
    for c in feat_to_convert:
        data[c] = pd.to_datetime(data[c]).dt.date
    return data

In [16]:
feat_to_convert = ["order_purchase_timestamp",
                   "review_creation_date",
                   "review_answer_timestamp"]
df = date_monthly(df, feat_to_convert)

In [17]:
# export in csv format the merge dataset
df.to_csv("data/data.csv")

## Ingénierie des variables

Bon nombre des variables ne sont pas exploitables en l'état, il va falloir les transformer avant de penser à les utiliser pour faire tourner des modèles de partionnement.

De plus, ayant pour but de faire une segmentation des clients il va nous falloir regrouper les données par client. Dans la base de donnée, un client unique est assigné à chaque transaction. Ainsi, un client ayant effectué plusieurs transactions se verra attribuer plusieurs *customer_id*. C'est pourquoi nous avons aussi accès à la variable *customer_unique_id*. C'est cette dernière que nous utiliserons pour regrouper les données.

### Variables RFM

La méthode de segmentation de clients la plus utilisée est le calcul d'un score RFM (Recency, Frequency, Monetary Value). Ce score consiste à donner une note entre 1 et 10 pour trois variables :
- récence : maximum entre "10 - le nombre de mois écoulés entre aujourd'hui et le dernier achat effectué" et 1
- fréquence : maximum entre "le nombre d'achats effectués (avec une limite de 10) sur une période (en général 12 mois) et 1".
- achat : totalité des achats sur une période de temps ou montant du panier moyen. Ici nous prendrons le panier moyen, et attribuerons comme note le numéro du décile.

Le score est obtenu en additionnant ces trois notes.

In [18]:
def get_rfm(data, m_mean=False):
    """Function to get the three RFM features,
    commonly used in database marketing.
    Parameters:
    data: DataFrame
        the pandas object holding data
    m_mean: bool, default False
        to get the mean and not the total for the monetary value
    -----------
    Return:
        DataFrame
    """
    r_feature = "order_purchase_timestamp"
    f_feature = "order_id"
    m_feature = "price"
    # Set a reference date
    date_ref = data[r_feature].max()
    # get the Recency
    recency = (data.groupby("customer_unique_id")
                   .order_purchase_timestamp
                   .max())
    recency = 10 - (date_ref-recency)/np.timedelta64(1, 'M')
    recency = (recency.round()
                      .fillna(0)
                      .where(recency>1, other=1))
    # get the Monetary_value
    monetary = (data.groupby("customer_unique_id")
                    .price
                    .sum()
                    .fillna(0))
    if m_mean:
        monetary /= (data.groupby("customer_unique_id")
                         .order_id
                         .nunique())
    monetary = (pd.qcut(monetary, q=10,
                        labels=np.linspace(1, 10, 10))
                  .astype(int))
    # get the Frequency
    frequency = (data.groupby("customer_unique_id")
                     .order_id
                     .nunique())
    frequency = (frequency.fillna(0)
                          .where(frequency>0, other=1)
                          .where(frequency<10, other=10))
    # Create the DataFrame
    df = pd.DataFrame({"Recency": recency,
                       "Frequency": frequency,
                       "Monetary_value": monetary},
                      index=recency.index)
    return df

In [19]:
data = get_rfm(df, m_mean=True)

### Nombre de produits par panier

En plus du RFM, il peut être utilé de connaître le nombre moyen de produits contenus dans le panier. Cela permettrait notamment de distinguer les grossistes des particuliers.

In [20]:
def products_per_order(data):
    """Function to get the average number
    of products per order.
    Parameters:
    data: DataFrame
        the pandas object holding data
    -----------
    Return: Series
        the pandas object holding data
    """
    s = data.groupby("customer_unique_id")["product_id"].count()/ \
        data.groupby("customer_unique_id")["order_id"].nunique()
    return s.fillna(0)

In [21]:
data["order_n_products"] = products_per_order(df)

### Variables liées aux produits

Nous allons maintenant essayer de déterminer le produit type acheté par chaque client en ajoutant trois variables à notre DataFrame :
- prix
- longeur de la description
- frais de port

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

def product_type(data):
    """Function to get the type product for each customer.
    Parameters:
    data: DataFrame
        the pandas object holding data
    -----------
    Return: 1d Array
        the numpy object holding data
    """
    # create a dataframe will all required features
    features = ["price",
                "product_description_lenght",
                "freight_value"]
    
    df = data.groupby("customer_unique_id")[features].mean()
    # Impute missing data by the mean
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    X = imp_mean.fit_transform(df.values)

In [23]:
data["product_type"] = product_type(df)

### Catégories des produits

Nous allons maintenant créer une variable par catégorie de produit que nous renseignerons par la proportion de produits de cette catégorie acheté par chaque client. C'est la version anglaise des noms de catégorie qui sera utilisé.

Puis nous réaliserons une ACP de manière à voir si nous pouvons réduire ce nombre de catégorie.

In [24]:
def product_category(data):
    """Function to get the product category for each customer.
    Parameters:
    data: DataFrame
        the pandas object holding data
    -----------
    Return: 1d Array
        the numpy object holding data
    """
    df = data.copy()
    # Create a feature by category
    list_products_cat = df["product_category_name_english"].unique().tolist()
    for c in list_products_cat:
        cond = df["product_category_name_english"]==c
        df[c] = 1
        df[c] = df[c].where(cond, 0)
    # Group all data by customer
    df = df.groupby("customer_unique_id")[list_products_cat].sum()
    # Calculate the proportions
    for c in list_products_cat:
        df[c] /= (data.groupby("customer_unique_id")
                      .product_id
                      .count())
    # Impute missing data and normalize data
    X = df.fillna(0).values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    # Realize the PCA
    pca = PCA(n_components=1)
    pca.fit(X_scaled)
    print("The percentage of variance explained by the first component is:\n \
          {}".format(pca.explained_variance_ratio_))
    return pca.transform(X_scaled)

In [25]:
data["product_category"] = product_category(df)

The percentage of variance explained by the first component is:
           [0.01486604]


Nous constatons que la proportion de la variance expliquée par le première composante est trés faible. Ceci nous indique que chaque variable est fortement décorrélée des autres et donc que l'ACP n'arrive pas à les regrouper. Dans ces conditions, l'ACP nous sera d'aucune utilité.

Nous pourrions envisager de ne garder que les catégories les plus achetées, mais se poserait alors la question de combien de catégorie garder. Les résultats de notre clustering seront fort probablement différents selon que nous décidions de garder X ou Y variables, et donc biaiserait les résultats.

Pour ces raisons, nous n'utiliserons pas les catégories pour effectuer clustering. En revanche, nous les utiliserons une fois le partitionnement effectué afin de caractériser les clusters.

### Variables liées aux commentaires

Nous traiterons maintenant les variables liées aux commentaires. Pour chaque client, nous calculerons un score (entre 1 et 10) pour les variables suivantes :
- temps de réponse moyen
- nombre de commentaires
- score moyen des commentaires

Puis nous additionnerons ces 3 scores, et diviserons le résultat par 3 pour avoir une note finale entre 1 et 10.

In [26]:
def review_score(data):
    """Function to get a score of the reviews
    based on three features:
    - the satisfaction survey answer timedelta
    - number of reviews per customer
    - the mark given by the review
    Parameters:
    data: DataFrame
        the pandas object holding data
    -----------
    Return: Series
        the pandas object holding data
    """
    df = data.copy()
    # get the survey answer timedelta in days
    df["review_answer_timedelta"] = df["review_answer_timestamp"]
    df["review_answer_timedelta"] -= df["review_creation_date"]
    df["review_answer_timedelta"] /= np.timedelta64(1, 'D')
    review_timedelta = (df.groupby("customer_unique_id")
                          .review_answer_timedelta
                          .mean())
    review_timedelta = 10 - review_timedelta
    review_timedelta = (review_timedelta.round()
                                        .fillna(0)
                                        .where(review_timedelta>1,
                                               other=1))
    # get the number of review per customer
    review_n = (df.groupby("customer_unique_id")
                  .review_id
                  .count())
    review_n = (review_n.fillna(0)
                        .where(review_n>0, other=1)
                        .where(review_n<10, other=10))
    # get the review mark
    review_mark = (df.groupby("customer_unique_id")
                     .review_score
                     .mean())
    review_mark *=2
    review_mark = (review_mark.fillna(review_mark.mean())
                              .round())
    # get the score
    review = review_timedelta + review_n + review_mark
    review /= 3
    return review.round()

In [27]:
data["review"] = review_score(df)

### Moyens de paiement

Il pourrait être utile de voir le moyen de paiement privilégié par chaque client. Pour cela nous allons créer une variable par moyen de paiement possible, et nous renseignerons la proportion du montant total payé par ce moyen de paiement.

Commençons par vérifier les types de paiement possible.

In [28]:
df.payment_type.value_counts()

credit_card    87784
boleto         23190
voucher         6465
debit_card      1706
not_defined        3
Name: payment_type, dtype: int64

Nous remarquons que les trois dernière modalités regroupent moins de 10% des transactions, nous allons les regrouper sous la catégorie *other_payment*.

In [29]:
def payment_type(data):
    """Function to get the type of payment used by each customer.
    Parameters:
    data: DataFrame
        the pandas object holding data
    -----------
    Return: tuple
        * The list of the payment_types
        * the numpy object holding data
    """
    df = data.copy()
    df.payment_type.where(df.payment_type.isin(["credit_card", "boleto"]),
                          "other_payment", inplace=True)
    # Create a feature by type of payment
    list_payment_type = (df.payment_type
                           .unique()
                           .tolist())
    for c in list_payment_type:
        cond = df["payment_type"]==c
        df[c] = df["payment_value"]
        df[c] = df[c].where(cond, 0)
    # Group all data by customer
    df = df.groupby("customer_unique_id")[list_payment_type].sum()
    # Calculate the proportions
    for c in list_payment_type:
        df[c] /= (data.groupby("customer_unique_id")
                      .payment_value
                      .sum())
    return list_payment_type, df.fillna(0).values

In [30]:
list_payment, payment_array = payment_type(df)
for i, c in enumerate(list_payment):
    data[c] = payment_array[:, i]

## Mise à l'échelle des données

Nous allons créer une fonction pour changer l'échelle des données. Nous commencerons par normaliser les données. Puis, nous changerons le poids des variables *RFM* de manière à ce que ces trois dernières représentent la moitié du poids total des variables.

In [31]:
from sklearn.preprocessing import StandardScaler

def scale_data(data):
    """Function to scale data by:
    1) removing the mean and scaling to unit variance.
    2) Increasing the weigh of RFM features
    -----------
    Parameters:
    data: DataFrame
        the pandas object holding data
    -----------
    Return:
        DataFrame
    """
    rfm_feat = ["Recency", "Frequency", "Monetary_value"]
    df = data.copy()
    n = data.columns.size
    weight = (n-3) / 3
    X = data.values
    std_scaler = StandardScaler()
    X_scaled = std_scaler.fit_transform(X)
    df.loc[:, :] = X_scaled
    df.loc[:, rfm_feat] *= weight
    return df

## Création d'un pipeline

Pour finir nous créerons un pipeline permettant de faire toutes ces transformations à la suite. Nous ajouterons en amont du pipeline une fonction permettant de filtrer les données en ne gardant que les transactions antérieures à une date de référence qui sera entrée en paramètre.

In [32]:
def data_filter(data, ref_date):
    """Function to filter data by keeping only
    the orders purchased prior to a reference data
    entered in parameter.
    Parameters:
    data: DataFrame
        the pandas object holding data
    ref_date: datetime object
        the reference date to be used to filter data
    -----------
    Return:
        DataFrame
    """
    return data[data.order_purchase_timestamp<=ref_date]

In [33]:
def wrangling_pipeline(data, ref_date=None, m_mean=False):
    """pipeline to carry out all functions of data wrangling.
    Parameters:
    data: DataFrame
        the pandas object holding data
    ref_date: datetime object
        the reference date to be used to filter data    
    m_mean: bool, default False
        to get the mean and not the total for the monetary value
    -----------
    Return:
        DataFrame
    """
    if not ref_date:
        ref_date = data.order_purchase_timestamp.max()
    df = data_filter(data, ref_date=ref_date)
    df = get_rfm(df, m_mean=m_mean)
    df["order_n_products"] = products_per_order(data)
    df["product_type"] = product_type(data)
    df["product_category"] = product_category(data)
    df["review"] = review_score(data)
    list_payment, payment_array = payment_type(df)
    for i, c in enumerate(list_payment):
        df[c] = payment_array[:, i]
    return scale_data(df)